<img src='../img/logo.png' alt='DS Market logo' height='150px'>

# Clustering

## Table of Contents

* [A. Introduction](#introduction)
* [B. Importing Libraries](#libraries)
* [C. Importing data](#data)
* [D. Products Clustering](#products)
* [E. Store Clustering](#store_clustering)

## A. Introduction <a class="anchor" id="introduction"></a>

The goal of this notebook is to identify groups of products that behave in a similar way, so DS Market can evaluate the performance of their different campaigns. We will provide those groups and the ideal number of these.

Additionally, we will try to identify how similar are stores from one another in case it makes any sense to do it.

## B. Importing Libraries <a class="anchor" id="libraries"></a>

In [1]:
# system and path management
import sys
sys.path.append('../scripts') # including helper functions inside the scripts folder

# removing system warnings
import warnings
warnings.filterwarnings('ignore')

# data manipulation
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# plotting
import matplotlib.pyplot as plt
import plotly.express as px

# plotting options
%matplotlib inline
plt.style.use('ggplot')
plt.rcParams["figure.figsize"] = (10, 7)

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.options.display.float_format = '{:,.2f}'.format

# helper functions
import file_management
import clustering

## C. Importing Data <a class="anchor" id="data"></a>

In [2]:
# downloading the processed data files from gdrive
directory = '../data/processed/'
urls = [
    {'filename': 'sales_processed.csv', 'url': 'https://drive.google.com/file/d/1JdeAgraKcaFQJrjG2HPVb5D0VD0iTlNB/view?usp=sharing'},
    {'filename': 'prices_processed.csv', 'url': 'https://drive.google.com/file/d/1pSEJAQfAU-owDjKmxcPrxf3CpGFivwa6/view?usp=sharing'},
    {'filename': 'calendar_processed.csv', 'url': 'https://drive.google.com/file/d/1Lnji96iBkTpFiWo-QXeW3TvESiNYWCML/view?usp=sharing'}
]
        
file_management.download_files_from_url(urls, directory)

sales = pd.read_csv(directory + 'sales_processed.csv', index_col = 0)
prices = pd.read_csv(directory + 'prices_processed.csv', index_col = 0)
calendar = pd.read_csv(directory + 'calendar_processed.csv', index_col = 0)

sales_processed.csv file already exists in ../data/processed/
prices_processed.csv file already exists in ../data/processed/
calendar_processed.csv file already exists in ../data/processed/


In [3]:
# downloading the feature files from gdrive
directory = '../data/features/'
urls = [
    {'filename': 'sales_by_date.csv', 'url': 'https://drive.google.com/file/d/1JMy2pJUp7DscjnY3_vhCNM7NZk9Th4i9/view?usp=sharing'},
    {'filename': 'sales_by_date_store.csv', 'url': 'https://drive.google.com/file/d/17Na9Eyj_NUGt9Uial1Oepwn8neUTXMmp/view?usp=sharing'},
    {'filename': 'sales_by_date_city.csv', 'url': 'https://drive.google.com/file/d/1Psykw5DZ7JfQkHYcajVW2ZlcaYoj2rmd/view?usp=sharing'},
    {'filename': 'sales_by_product.csv', 'url': 'https://drive.google.com/file/d/1bayt13OJ8NFjfYkscPac6J2HiuyrSqEK/view?usp=sharing'}
]

file_management.download_files_from_url(urls, directory)

sales_by_date = pd.read_csv(directory + 'sales_by_date.csv', index_col = 0)
sales_by_date_store = pd.read_csv(directory + 'sales_by_date_store.csv', index_col = 0)
sales_by_date_city = pd.read_csv(directory + 'sales_by_date_city.csv', index_col = 0)
sales_by_product = pd.read_csv(directory + 'sales_by_product.csv', index_col = 0)

sales_by_date.csv file already exists in ../data/features/
sales_by_date_store.csv file already exists in ../data/features/
sales_by_date_city.csv file already exists in ../data/features/
sales_by_product.csv file already exists in ../data/features/


## D. Product Clustering<a class="anchor" id="product_clustering"></a>

Let's explore how if our products behave in similar ways. A first approach would be to see how they when we confront their average price and the sales average from the data we have.  

In [4]:
df = pd.DataFrame({
    'avg_sell_price' : sales_by_product.iloc[2:, 3],
    'avg_num_sales': sales_by_product.iloc[2:, 1],
    'std_sell_price': sales_by_product.iloc[2:, -1]
})

df['avg_sell_price'] = df['avg_sell_price'].astype('float')
df['avg_num_sales'] = df['avg_num_sales'].astype('float')
df['std_sell_price'] = df['std_sell_price'].astype('float')

df

,avg_sell_price,avg_num_sales,std_sell_price
ACCESORIES_1_001,"219,344.58",0.21,0.73
ACCESORIES_1_002,"100,941.36",0.26,0.09
ACCESORIES_1_003,"75,518.03",0.08,0.13
ACCESORIES_1_004,"114,391.52",2.05,0.28
ACCESORIES_1_005,"73,414.40",0.76,0.22
...,...,...,...
SUPERMARKET_3_823,"63,984.10",0.80,0.23
SUPERMARKET_3_824,"57,897.00",0.44,0.26
SUPERMARKET_3_825,"94,370.36",0.71,0.24
SUPERMARKET_3_826,"29,381.33",0.64,0.01


In [12]:
px.scatter_3d(
    df, 
    x ='avg_sell_price', 
    y = 'avg_num_sales', 
    z = 'std_sell_price', 
    color = df.index, 
    title = 'Scatter plot  - Product AVG Prices VS Product AVG Sales',
    height = 800
)

In [9]:
# scaling data
scaler = StandardScaler()
scaled_selected_data = scaler.fit_transform(df)
scaled_df = pd.DataFrame(scaled_selected_data, columns = df.columns, index = df.index)

In [10]:
px.scatter(scaled_df, x ='avg_sell_price', y = 'avg_num_sales', color = df.index, title = 'Scatter plot  - Product AVG Prices VS Product AVG Sales (Scaled)')

In [8]:
# elbow method to identify the right amount of clusters
clustering.kmeans_elbow_plot(scaled_selected_data, (1, 12))

## E. Store Clustering<a class="anchor" id="store_clustering"></a>

____

clustering:

- scatter plot... 1v1 (with num variables)
- what caracterises a store? amount of sales they have? Num of products they have...
